In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import os
import cv2
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

In [2]:
brooke_df = pd.read_csv('brooke_df.csv')
df1 = brooke_df.drop_duplicates(subset=['Subj#'], keep='first').reset_index(drop=True)
df1['Group'].value_counts()

Group
DMD        28
Healthy    23
SMA        12
Name: count, dtype: int64

In [3]:
label_df = pd.read_csv('label_df.csv')
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Subj#       146 non-null    int64 
 1   Visit Date  146 non-null    object
 2   rflex1_s    146 non-null    object
 3   rflex1_e    146 non-null    object
 4   rflex2_s    146 non-null    object
 5   rflex2_e    146 non-null    object
 6   rext1_s     146 non-null    object
 7   rext1_e     146 non-null    object
 8   rext2_s     146 non-null    object
 9   rext2_e     146 non-null    object
 10  lflex1_s    146 non-null    object
 11  lflex1_e    146 non-null    object
 12  lflex2_s    146 non-null    object
 13  lflex2_e    146 non-null    object
 14  lext1_s     146 non-null    object
 15  lext1_e     146 non-null    object
 16  lext2_s     146 non-null    object
 17  lext2_e     146 non-null    object
dtypes: int64(1), object(17)
memory usage: 20.7+ KB


In [34]:
label_df

,Subj#,Visit Date,rflex1_s,rflex1_e,rflex2_s,rflex2_e,rext1_s,rext1_e,rext2_s,rext2_e,lflex1_s,lflex1_e,lflex2_s,lflex2_e,lext1_s,lext1_e,lext2_s,lext2_e
0,1,1/27/2021,00:03:11.776,00:03:13.029,00:03:15.092,00:03:16.363,00:03:13.761,00:03:15.025,00:03:16.429,00:03:17.631,00:03:11.780,00:03:13.010,00:03:15.100,00:03:16.360,00:03:13.740,00:03:15.010,00:03:16.420,00:03:17.620
1,1,7/8/2021,00:02:30.870,00:02:32.158,00:02:33.613,00:02:34.528,00:02:32.190,00:02:33.480,00:02:34.561,00:02:35.531,00:02:30.870,00:02:32.150,00:02:33.600,00:02:34.510,00:02:32.180,00:02:33.480,00:02:34.560,00:02:35.530
2,1,11/10/2021,00:02:39.012,00:02:40.145,00:02:41.612,00:02:42.778,00:02:45.200,00:02:46.366,00:02:40.178,00:02:41.545,00:02:39.010,00:02:40.130,00:02:41.620,00:02:42.770,00:02:42.810,00:02:43.950,00:02:40.170,00:02:41.540
3,1,8/15/2023,00:05:02.560,00:05:03.635,00:04:57.930,00:04:59.080,00:05:03.669,00:05:04.693,00:04:59.144,00:05:00.104,00:09:26.510,00:09:27.612,00:09:22.768,00:09:23.505,00:09:25.655,00:09:26.477,00:09:23.539,00:09:24.472
4,1,2/20/2024,00:01:32.278,00:01:33.078,00:01:34.044,00:01:34.911,00:01:33.078,00:01:33.811,00:01:34.978,00:01:35.978,00:03:28.016,00:03:29.150,00:03:30.783,00:03:31.550,00:03:29.216,00:03:30.683,00:03:31.783,00:03:32.683
5,1,8/28/2024,00:01:32.278,00:01:33.078,00:01:34.044,00:01:34.911,00:01:33.078,00:01:33.811,00:01:34.978,00:01:35.978,00:03:28.016,00:03:29.150,00:03:30.783,00:03:31.550,00:03:29.216,00:03:30.683,00:03:31.783,00:03:32.683
6,2,2/11/2021,00:03:26.860,00:03:29.482,00:03:12.576,00:03:16.321,00:03:23.534,00:03:25.267,00:03:16.387,00:03:18.062,00:07:29.868,00:07:33.068,00:07:34.711,00:07:35.744,00:07:33.068,00:07:34.701,00:07:35.744,00:07:37.278
7,3,2/24/2021,00:02:45.595,00:02:46.828,00:02:43.295,00:02:44.195,00:02:44.428,00:02:45.495,00:02:46.840,00:02:47.407,00:05:41.546,00:05:42.313,00:05:39.313,00:05:40.080,00:05:40.313,00:05:41.480,00:05:42.413,00:05:43.146
8,3,7/20/2021,00:04:27.349,00:04:28.082,00:04:29.116,00:04:30.016,00:04:30.282,00:04:31.149,00:04:28.149,00:04:29.016,00:04:21.455,00:04:22.255,00:04:23.721,00:04:24.821,00:04:24.830,00:04:25.464,00:04:22.355,00:04:23.455
9,3,1/26/2022,00:10:19.385,00:10:20.552,00:10:15.107,00:10:15.840,00:10:20.552,00:10:21.452,00:10:15.946,00:10:16.780,00:06:23.876,00:06:24.876,00:05:03.008,00:05:03.775,00:06:24.876,00:06:26.010,00:05:03.908,00:05:05.008


In [23]:
def extract_activity_segments(row):
    segments = []
    for col in row.index:
        if col.endswith('_s'):
            activity = col[:-2]  # strip _s
            start_time = datetime.strptime(row[col], '%H:%M:%S.%f')
            end_col = activity + '_e'
            end_time = datetime.strptime(row[end_col], '%H:%M:%S.%f')
            segments.append((activity, start_time, end_time))
    return segments

def to_seconds(dt):
    return dt.hour * 3600 + dt.minute * 60 + dt.second + dt.microsecond / 1e6


def segment_video(input_path, output_path, start_sec, end_sec):
    cap = cv2.VideoCapture(str(input_path))
    if not cap.isOpened():
        raise IOError(f"Cannot open video file {input_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    start_frame = int(start_sec * fps)
    end_frame = int(end_sec * fps)

    if start_frame >= total_frames or end_frame > total_frames:
        raise ValueError(f"Frame range ({start_frame}-{end_frame}) exceeds total frames ({total_frames})")

    # Set codec and create VideoWriter
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    for frame_num in range(start_frame, end_frame):
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()

In [28]:
video_dir = Path("videos")
output_dir = Path("videos")

for index, row in label_df.iterrows():
    subj = str(row['Subj#']).strip()
    visit_date = str(row['Visit Date']).strip()
    visit_date_clean = pd.to_datetime(visit_date).strftime('%m-%d-%y')
    video_path = Path(os.path.join(video_dir, subj, visit_date_clean))
    mp4_file = list(video_path.glob("*.mp4"))[0]
    print(mp4_file)
          
    segments = extract_activity_segments(row)
    segments_in_seconds = [(act, to_seconds(start), to_seconds(end)) for act, start, end in segments]

    out_dir = video_path / "segments"
    out_dir.mkdir(parents=True, exist_ok=True)

    for act, start_sec, end_sec in segments_in_seconds:
        act_root = ''.join(filter(str.isalpha, act))  # e.g., 'rflex1' -> 'rflex'
        act_num = ''.join(filter(str.isdigit, act))   # e.g., 'rflex1' -> '1'
        output_path = out_dir / f"{subj}_{act_root}_{act_num}.mp4"
        segment_video(mp4_file, output_path, start_sec, end_sec)

videos/1/01-27-21/color1616769800710_x264.mp4
videos/1/07-08-21/color1625759427380_x264.mp4
videos/1/11-10-21/color1636561983720_x264.mp4
videos/1/08-15-23/color1692121528321_x264.mp4
videos/1/02-20-24/color1724862541872_x264.mp4
videos/1/08-28-24/color1724862541872_x264.mp4
videos/2/02-11-21/color1613062261963_x264.mp4
videos/3/02-24-21/color1614197603948_x264.mp4
videos/3/07-20-21/color1626796633372_x264.mp4
videos/3/01-26-22/color1643218466934_x264.mp4
videos/5/03-11-21/color1615480092322_x264.mp4
videos/7/04-21-21/color1619029821700_x264.mp4
videos/8/04-21-21/color1619027638219_x264.mp4
videos/9/03-24-21/color1697129357562_x264.mp4
videos/9/09-22-21/color1697129357562_x264.mp4
videos/9/03-23-22/color1648043865590_x264.mp4
videos/9/09-20-22/color1663693031221_x264.mp4
videos/9/03-22-23/color1697129357562_x264.mp4
videos/9/10-12-23/color1697129357562_x264.mp4
videos/10/03-24-21/color1616603218888_x264.mp4
videos/10/06-09-21/color1731609682625_x264.mp4
videos/10/05-12-22/color17316096